In [1]:
import cv2
import numpy as np
import tensorflow as tf
import winsound  # For playing a beep sound (Windows only)

# Load the pre-trained model
new_model = tf.keras.models.load_model('fatigue_model_cnn.h5')

# Load Haar cascade classifiers for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize video capture
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Initialize variables for consecutive sleepy frames
sleepy_count = 0
sleepy_threshold = 8  # Set threshold to 5 consecutive "Sleepy" frames
alarm_triggered = False  # Flag to track if the alarm has been triggered

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Initialize status
    status = "Unknown"

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        # Detect eyes within the face region
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 4)

        if len(eyes) > 0:  # If eyes are detected
            for (ex, ey, ew, eh) in eyes:
                roi_eye_color = roi_color[ey:ey + eh, ex:ex + ew]

                # Resize the detected eye region and prepare for prediction
                try:
                    final_image = cv2.resize(roi_eye_color, (224, 224))
                    final_image = np.expand_dims(final_image, axis=0)  # Add batch dimension

                    # Normalize image (if your model expects this)
                    final_image = final_image / 255.0  # If the model was trained with this normalization

                    # Make predictions
                    Predictions = new_model.predict(final_image)

                    # Get predicted class
                    predicted_class = np.argmax(Predictions, axis=1)[0]
                    confidence = np.max(Predictions)  # Get the confidence level

                    # Determine status based on predicted class
                    if predicted_class == 0:  # Assuming class 0 corresponds to "Active"
                        status = f"Active ({confidence * 100:.2f}%)"
                        if alarm_triggered:
                            # Reset alarm and threshold when status becomes Active
                            sleepy_count = 0
                            alarm_triggered = False  # Stop the alarm
                        else:
                            sleepy_count = 0  # Reset the sleepy count if active
                    elif predicted_class == 1:  # Assuming class 1 corresponds to "Sleepy"
                        status = f"Sleepy ({confidence * 100:.2f}%)"
                        sleepy_count += 1  # Increment sleepy count

                except Exception as e:
                    print(f"Error in processing: {e}")
                    status = "Error"

        else:
            # If no eyes detected, mark as "Sleepy"
            status = "Sleepy"
            sleepy_count += 1  # Increment sleepy count if no eyes detected

        # If the number of consecutive "Sleepy" frames exceeds the threshold, trigger alarm
        if sleepy_count >= sleepy_threshold and not alarm_triggered:
            winsound.Beep(1000, 1000)  # Beep sound (frequency 1000Hz, duration 1000ms)
            alarm_triggered = True  # Set the flag to indicate that the alarm has been triggered
            sleepy_count = 0  # Reset the count after the beep

        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Display status text on video
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, status, (50, 50), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow('Sleepiness Detection Window', frame)

    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━